# A Neural Engineering Framework Representation of the Kalman Filter
## Introduction
The Kalman Filter is a method for estimating unknown or hidden system states using an internal model of the physical processes at work. First we need to construct the physical process that we are trying to find estimates for. Modelling kinematic motion can be done through state space equation in discrete time using simple forward integration. For system dynamics in the horizontal axis of the 2D plane let $$x[n+1] = x[n] + \dot{x} * dt$$ and system dynamics in the vertical axis be $$y[n+1] = y[n] + \dot{y}[n] * dt$$ where $$\dot{y}[n+1] = \dot{y}[n] + \ddot{y} * dt$$ which gives $$y[n+1] = y[n] + \dot{y}[n-1] * dt + \ddot{y} * dt^2$$ Written in a state space form this gives: 

In [4]:
import nengo
import numpy as np

In [3]:
def system_dynamics(x, t):
    
    
    return dxdt
    

In [ ]:
t_length = 5
t_space = np.linspace(0, t_length, num=50)

x_i = 10
y_i = 0
x_dot_i = 3
y_dot_i = 3

input_signal = np.array([])